In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
titanic = sns.load_dataset('titanic')

titanic = titanic.copy()
titanic = titanic.dropna()

titanic['age'].plot.hist(
  bins = 50,
  title = "Histogram of the age variable"
)

# Z-Score

In [ ]:
from scipy.stats import zscore

titanic["age_zscore"] = zscore(titanic["age"])

titanic["is_outlier"] = titanic["age_zscore"].apply(
  lambda x: x <= -2.5 or x >= 2.5
)
titanic[titanic["is_outlier"]]

# DBSCAN — Density-Based Spatial Clustering of Applications with Noise

In [ ]:
ageAndFare = titanic[["age", "fare"]]
ageAndFare.plot.scatter(x = "age", y = "fare")

Ölçeklendirme gerekli
> Her iki değişkenin de farklı ölçeklere sahip olduğunu görüyoruz.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
ageAndFare = scaler.fit_transform(ageAndFare)
ageAndFare = pd.DataFrame(ageAndFare, columns = ["age", "fare"])
ageAndFare.plot.scatter(x = "age", y = "fare")

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(
  eps = 0.5,
  metric="euclidean",
  min_samples = 3,
  n_jobs = -1)
model = dbscan.fit(ageAndFare)

labels = model.labels_
labels

# Çekirdek örnekleri (Core samples)

In [ ]:
core_samples = np.zeros_like(labels, dtype = bool)
core_samples[dbscan.core_sample_indices_] = True
core_samples

# Küme Sayısı

In [ ]:
len(set(labels) - set([-1]))

# Değerlendirme

## Siluet Puanı (Silhouette Score):
[-1, 1] arasında.
> birbirine çok yakın (yüksek yoğunluklu) çok sayıda veri noktasına sahip bir kümenin oluşması ve bir sonraki en yakın kümeden uzakta olması durumunda (kümenin bir sonraki en yakın kümeye kıyasla çok benzersiz olduğunu gösterir), güçlü bir siluete sahip olacaktır


In [ ]:
from sklearn import metrics
metrics.silhouette_score(ageAndFare, labels)

In [ ]:
plt.plot(ageAndFare[labels > -1].age, ageAndFare[labels > -1].fare, 'go')
plt.plot(ageAndFare[labels == -1].age, ageAndFare[labels == -1].fare, 'rx')
plt.legend(["Clusters", "anomalies"])